In [1]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [2]:
tags = ['gent', 'datv', 'accs', 'ablt', 'loct', 'plur']

#все возможные формы существительных
def inflect(word, morph, tags, i):
    nouns_inflected = []
    noun_parsed = morph.parse(word)[i]
    for tag in tags:
        try:
            n_sing = noun_parsed.inflect({tag})[0]
            nouns_inflected.append(n_sing)
            n_plur = noun_parsed.inflect({'plur', tag})[0]
            nouns_inflected.append(n_plur)
        except:
            pass
    return list(set(nouns_inflected))

#для омонимичных форм (pymorphy разбирает "начало", в первую очередь, как глагол, только потом существительное)
def get_only_nouns(word, morph):
    noun_parsed = morph.parse(word)[0]
    if "NOUN" in noun_parsed.tag:
        try:
            nouns = inflect(word, morph, tags, 0)
            return nouns
        except:
            pass
    else:
        try:
            nouns = inflect(word, morph, tags, 1)
            return nouns
        except:
            pass

In [3]:
s = open('C:/Users/1/Desktop/freqrnc2011.csv', 'r', encoding='utf-8')
lines = s.readlines()

In [4]:
#s - обозначение существительного в частотном словаре
#неначальные формы получают частоту начальной
dic = {}

for line in lines:
    tokens = line.split("\t")
    if tokens[1] == "s":
        tokens[0] = tokens[0].strip('"')
        if tokens[0] not in dic.keys():
            dic[tokens[0]] = float(tokens[2])
        else:
            dic[tokens[0]] += float(tokens[2])
        if get_only_nouns(tokens[0], morph) != None:
            for item in get_only_nouns(tokens[0], morph):
                if item not in dic.keys():
                    dic[item] = float(tokens[2])

In [5]:
dic

{'а': 2.7,
 'абажур': 4.9,
 'абажуру': 4.9,
 'абажурах': 4.9,
 'абажуры': 4.9,
 'абажурам': 4.9,
 'абажурами': 4.9,
 'абажуре': 4.9,
 'абажуров': 4.9,
 'абажура': 4.9,
 'абажуром': 4.9,
 'аббат': 1.6,
 'аббате': 1.6,
 'аббатами': 1.6,
 'аббатам': 1.6,
 'аббаты': 1.6,
 'аббата': 1.6,
 'аббатом': 1.6,
 'аббатах': 1.6,
 'аббату': 1.6,
 'аббатов': 1.6,
 'аббатство': 0.9,
 'аббатствами': 0.9,
 'аббатстве': 0.9,
 'аббатствах': 0.9,
 'аббатству': 0.9,
 'аббатством': 0.9,
 'аббатствам': 0.9,
 'аббатства': 0.9,
 'аббатств': 0.9,
 'аббревиатура': 3.5,
 'аббревиатур': 3.5,
 'аббревиатуру': 3.5,
 'аббревиатуры': 3.5,
 'аббревиатуре': 3.5,
 'аббревиатурах': 3.5,
 'аббревиатурой': 3.5,
 'аббревиатурам': 3.5,
 'аббревиатурами': 3.5,
 'абвер': 2.7,
 'абверах': 2.7,
 'абверами': 2.7,
 'абверы': 2.7,
 'абвером': 2.7,
 'абвере': 2.7,
 'абверов': 2.7,
 'абверу': 2.7,
 'абверам': 2.7,
 'абвера': 2.7,
 'аберрация': 0.7,
 'аберрациями': 0.7,
 'аберрацией': 0.7,
 'аберрациях': 0.7,
 'аберраций': 0.7,
 'аберра

In [6]:
#все "ё" заменяются на "е" (в словаре "актер", а не "актёр")
dic_new = {}
for key, value in dic.items():
    if "ё" in key:
        key = key.replace("ё","е")
        dic_new[key] = value
    else:
        dic_new[key] = value

In [7]:
k = open('C:/Users/1/Desktop/dictionary_spell.tsv', 'w', encoding='utf-8')
for key, value in dic_new.items():
    k.write(key + "\t" + str(value) + "\n")
k.close()

В тестовом файле 10 ошибок, которые должен исправить спеллчекер: касметики, прадукт, рикламу, асобеностей, преставки, инцедентов, карпараций, сикунду, растаяние.

In [8]:
def P(word, N=sum(dic_new.values())): 
    return dic_new[word] / N

def correction(word): 
    return max(candidates(word), key=P)

def candidates(word): 
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    return set(w for w in words if w in dic.keys())

def edits1(word):
    letters    = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [9]:
v = open('C:/Users/1/Desktop/spellcheck_test.txt', 'r', encoding='utf-8')
test = v.read()
words_test = test.split()
nouns_test = []
for word in words_test:
    if word.islower() == True:
        word = word.strip(".,:;!?()")
        p = morph.parse(word)[0]
        if "NOUN" in p.tag:
            if any(str.isdigit(c) for c in word) == False:
                nouns_test.append(word.strip("()"))

In [10]:
for item in nouns_test:
    try:
        if correction(item) != item:
            print(item + " " + correction(item))
    except:
        pass

касметики косметики
прадукт продукт
годы году
рикламу рекламу
геймера гейзера
геймеров гейзеров
асобеностей особенностей
холодным холодом
любой людей
поп-звезд поп-звезда
преставки приставки
шутере шулере
инцедентов инцидентов
геймера гейзера
гигабайтах мегабайтах
трансформер транспортер
килограмма килограммы
гигабайтами мегабайтами
килограмма килограммы
карпараций корпораций
им имя
планетой планетов
планетой планетов
сикунду секунду
растаяние раскаяние


In [11]:
import Levenshtein

In [12]:
for noun in nouns_test:
    arr = []
    for key, value in dic_new.items():
        d = Levenshtein.distance(noun, key)
        arr.append((key, d))
    sorted_arr = sorted(arr, key=lambda tup: tup[1])

    arr_new = []
    for item in sorted_arr:
        if item[1] == sorted_arr[0][1]:
            arr_new.append((item[0], float(dic_new[item[0]])))
            
    arr_final = []
    if len(arr_new) == 1:
        if noun != sorted_arr[0][0]:
           print (noun + " " + sorted_arr[0][0])
    else:
        i = 0
        while i < len(arr_new) - 1:
            if float(arr_new[i+1][1]) > float(arr_new[i][1]):
                x = arr_new[i+1][0]
            else:
                x = arr_new[i][0]
            i += 1
        print (noun + " " + x)

супергерой суперменов
касметики косметики
прадукт продукт
годы ягоды
рикламу рекламу
геймера гейзера
геймеров гейзеров
героя-мужчину мужчину
асобеностей особенностей
слэшера эсера
мини-мюзикл мини-юбку
холодным холодком
любой юбкой
платформеров платформой
поп-звезд поп-звезда
преставки приставки
платформера-паззла платформах
людям-грибам любимицам
шутере шушере
инцедентов инцидентов
геймера гейзера
геймплея гейме
ноутбуков-трансформеров бронетранспортеров
гигабайтах мегабайтах
трансформер транспортер
килограмма килограммах
гигабайтами мегабайтами
килограмма килограммах
карпараций корпораций
им эм
планетой планеткой
планетой планеткой
магнитосферы магнитофоны
сикунду секунду
растаяние раскаяние
